In [3]:
using Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate();

  Activating project at `~/Desktop/Work/Research/LEGO-project/LEGO-3D_Dyn`


In [4]:
import ForwardDiff as FD
using LinearAlgebra
using Plots

include(joinpath(@__DIR__,"helpers/quaternions.jl"))

quat2rot (generic function with 1 method)

In [ ]:
function newton_euler_dynamics_with_gravity(
  q::Vector, 
  ᴮv::Vecotr, 
  ᴮω::Vector, 
  ᴮF::Vector, 
  ᴮτ::Vector, 
  params::NamedTuple)

  Q = quat2rot(q)
  m, J, g = params.m, params.J, params.g
  ᴺg = [0, 0, -g]
  ᴮg = Q' * ᴺg

  ᴺṙ = Q * ᴮv
  q̇ = (1/2) * L(q) * H * ᴮω
  ᴮv̇ = (1/m) * ᴮF + ᴮg - hat(ᴮω) * ᴮv
  ᴮω̇ = J \ (ᴮτ - hat(ᴮω) * (J * ᴮω))

  return [ᴺṙ; q̇; ᴮv̇; ᴮω̇]
end